# Merging HGDP and 1000 Genomes

This script will walk you through the merging of the HGDP and 1000 Genomes genotypes.
We will follow these steps:
- Download HGDP files, and transform them into a plink file
- Download 1000G files and keep only SNPs found in the HGDP files
- Merge the HGDP and 1000G files

## Needed packages

To run this script you'll need [plink](https://www.cog-genomics.org/plink2), [vcftools](https://vcftools.github.io/index.html), [bcftools](https://samtools.github.io/bcftools/bcftools.html).
All of these can be found in [bioconda](https://bioconda.github.io/)

## Preliminaries

First let's import modules and set up paths

In [1]:
import glob, os, shutil, subprocess, csv, time
import pandas as pd
import numpy as np

In [2]:
projpath = os.path.realpath("..")
pathhgdp = os.path.join(projpath, "DataBases", "HGDP")
path1000 = os.path.join(projpath, "DataBases", "1000G")
print(projpath)

/home/tomas/Documents/Repos/HGDP_1000G_Merge


## HGDP

The HGDP files (Stanford) were downloaded from [here](http://hagsc.org/hgdp/files.html), and the sample list file from [here](http://www.stanford.edu/group/rosenberglab/data/rosenberg2006ahg/SampleInformation.txt).
The script to transform the HGDP data to plink format is called HGDPtoPlink.sh and was modified from [here](http://www.harappadna.org/2011/02/hgdp-to-ped-conversion/).
The HGDP data uses coordinates from build 36.1 (a list of assemblies can be found [here](https://genome.ucsc.edu/FAQ/FAQreleases.html))

In [3]:
#Run the HGDPtoPlink script
#It can take a while
os.chdir(os.path.join(projpath, "Code"))
subprocess.run(["bash", "HGDPtoPlink.sh"])

dos2unix: converting file HGDP_FinalReport_Forward.txt to Unix format...
dos2unix: converting file HGDP_Map.txt to Unix format...
dos2unix: converting file SampleInformation.txt to Unix format...


PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hgdp.log.
Options in effect:
  --make-bed
  --missing-genotype -
  --out hgdp
  --output-missing-genotype 0
  --tfile hgdp

32108 MB RAM detected; reserving 16054 MB for main workspace.
Processing .tped file... done.2131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100%
hgdp-temporary.bed + hgdp-temporary.bim + hgdp-temporary.fam written.
660918 variants loaded from .bim file.
1043 people (673 males, 370 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1043 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646

treat these as missing.


1112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hgdp940.log.
Options in effect:
  --bfile hgdp
  --keep Sample_keep.txt
  --make-bed
  --out hgdp940

32108 MB RAM detected; reserving 16054 MB for main workspace.
660918 variants loaded from .bim file.
1043 people (673 males, 370 females) loaded from .fam.
--keep: 940 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 940 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyp

treat these as missing.


101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hgdp940.log.
Options in effect:
  --bfile hgdp940
  --out hgdp940
  --recode

32108 MB RAM detected; reserving 16054 MB for main workspace.
660918 variants loaded from .bim file.
940 people (618 males, 322 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 940 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.


treat these as missing.


Total genotyping rate is 0.998995.
660918 variants and 940 people pass filters and QC.
Note: No phenotypes present.
--recode ped to hgdp940.ped + hgdp940.map ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.


CompletedProcess(args=['bash', 'HGDPtoPlink.sh'], returncode=0)

Because the 1000G uses the GRCh37 assembly (fasta file can be found [here](http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/) as human_g1k_v37.fasta.gz) we'll need to liftover the HGDP coordinates.
To do that we'll use UCSC [liftOver](http://genome.ucsc.edu/cgi-bin/hgLiftOver) already installed using bioconda, and [liftOverPlink](https://github.com/sritchie73/liftOverPlink) as a wrapper to work with plink files (`ped` and `map` formats).
The chain file that tells liftOver how to convert between hg18 and hg19 can be downloaded [here](http://hgdownload.cse.ucsc.edu/goldenPath/hg18/liftOver/hg18ToHg19.over.chain.gz).

In [4]:
os.chdir(pathhgdp)
#Using liftover
liftoverplink = os.path.join(projpath, "Code", "liftOverPlink")
%run $liftoverplink --map hgdp940.map --out lifted --chain hg18ToHg19.over.chain.gz
badlifts = os.path.join(projpath, "Code", "rmBadLifts")
%run $badlifts --map lifted.map --out good_lifted.map --log bad_lifted.dat
#Creating a list of snps to include in lifted version
snps = pd.read_csv("good_lifted.map", sep = "\t", header = None)
snps.iloc[:,1].to_csv("snplist.txt", index = False)
#Excluding snps and creating binary file
subprocess.run(["plink", "--file", "hgdp940", "--recode", "--out", "lifted", "--extract", "snplist.txt" ])
subprocess.run(["plink", "--file", "--ped", "lifted.ped", "--map", "good_lifted.map", "--make-bed", "--out", "hgdp940hg19"])

#Removing some files
for file in glob.glob("*.ped"):
    os.remove(file)
    
for file in glob.glob("*.map"):
    os.remove(file)

Converting MAP file to UCSC BED file...
SUCC:  map->bed succ
Lifting BED file...


Reading liftover chains
Mapping coordinates


SUCC:  liftBed succ
Converting lifted BED file back to MAP...
SUCC:  bed->map succ
cleaning up BED files...
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to lifted.log.
Options in effect:
  --extract snplist.txt
  --file hgdp940
  --out lifted
  --recode

32108 MB RAM detected; reserving 16054 MB for main workspace.
.ped scan complete (for binary autoconversion).11111111111111212121212121212121213131313131313131314141414141414141415151515151515151515161616161616161616171717171717171717171818181818181818181919191919191919192020202020202020202021212121212121212122222222222222222222232323232323232323242424242424242424252525252525252525252626262626262626262727272727272727272728282828282828282829292929292929292930303030303030303030313131313131313131323232323232323232323333333333333333333434343434343434343535353535353535353536363636363636363637373737373737373737383838383838

In [5]:
#Create a snplist.bed with chr and position information
os.chdir(pathhgdp)
with open("snplist.bed", "w") as f:
    out = subprocess.run(["awk", "NR==FNR {h[$1] = 1; next} {if(h[$4]==1) print$0}", "snplist.txt", "snp151Common.bed"],
                         stdout=f, text=True)

In [6]:
#Read hgdp940hg19 log file
with open("hgdp940hg19.log") as myfile:
    for num, line in enumerate(myfile, 1):
        if "variants" in line:
            print(line, end='')
    print("Finished file... \n")

Performing single-pass .bed write (643948 variants, 940 people).
643948 variants loaded from .bim file.
643948 variants and 940 people pass filters and QC.
Finished file... 



In [7]:
dest_dir = os.path.join(projpath, "Results")
for filename in glob.glob("hgdp940hg19.*"):
    shutil.copy(filename, dest_dir)

## 1000G

The 1000G Phase 3 files were downloaded from [here](ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/).
First, we will extract the list of snps (`snplist.txt`) from the HGDP dataset for each chromosome of the 1000G samples using vcftools.
Then we will concatenate the different autosomal chromosomes in one file and convert it into a plink binary file using bcftools and plink.

In [10]:
#Extracting snps found in the HGDP to the 1000G files
#Takes time
os.chdir(path1000)
snplists = ["snplist.txt",
            "snplist.bed"]
for snpl in snplists:
    shutil.copy(os.path.join(pathhgdp, snpl), path1000)
for file in glob.glob("*chr[0-9]*.gz"):
    outname = file.split(".")[1] + "_extracted"
    subprocess.run(["vcftools", "--gzvcf", file, "--bed", "snplist.bed", "--recode", "--out", outname])


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
	--out chr21_extracted
	--recode
	--bed snplist.bed

Using zlib version: 1.2.11
After filtering, kept 2504 out of 2504 Individuals
Outputting VCF file...
	Read 652201 BED file entries.
After filtering, kept 9620 out of a possible 1105538 Sites
Run Time = 65.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
	--out chr17_extracted
	--recode
	--bed snplist.bed

Using zlib version: 1.2.11
After filtering, kept 2504 out of 2504 Individuals
Outputting VCF file...
	Read 652201 BED file entries.
After filtering, kept 16561 out of a possible 2329288 Sites
Run Time = 193.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf ALL.chr11.phase3

In [11]:
concatfiles = glob.glob("chr[0-9]*.recode.vcf")
function = ["bcftools", "concat", "-o", "1000g.vcf.gz", "-Oz"]
function.extend(concatfiles)
subprocess.run(function)

Checking the headers and starting positions of 16 files
Concatenating chr19_extracted.recode.vcf	1.328019 seconds
Concatenating chr22_extracted.recode.vcf	1.180856 seconds
Concatenating chr1_extracted.recode.vcf	5.801693 seconds
Concatenating chr16_extracted.recode.vcf	2.359901 seconds
Concatenating chr18_extracted.recode.vcf	2.431561 seconds
Concatenating chr20_extracted.recode.vcf	2.013835 seconds
Concatenating chr13_extracted.recode.vcf	2.970436 seconds
Concatenating chr2_extracted.recode.vcf	6.499817 seconds
Concatenating chr15_extracted.recode.vcf	2.397736 seconds
Concatenating chr14_extracted.recode.vcf	2.564049 seconds
Concatenating chr3_extracted.recode.vcf	5.352377 seconds
Concatenating chr11_extracted.recode.vcf	4.042636 seconds
Concatenating chr17_extracted.recode.vcf	2.128680 seconds
Concatenating chr10_extracted.recode.vcf	4.118487 seconds
Concatenating chr21_extracted.recode.vcf	1.264438 seconds
Concatenating chr12_extracted.recode.vcf	3.814989 seconds


CompletedProcess(args=['bcftools', 'concat', '-o', '1000g.vcf.gz', '-Oz', 'chr19_extracted.recode.vcf', 'chr22_extracted.recode.vcf', 'chr1_extracted.recode.vcf', 'chr16_extracted.recode.vcf', 'chr18_extracted.recode.vcf', 'chr20_extracted.recode.vcf', 'chr13_extracted.recode.vcf', 'chr2_extracted.recode.vcf', 'chr15_extracted.recode.vcf', 'chr14_extracted.recode.vcf', 'chr3_extracted.recode.vcf', 'chr11_extracted.recode.vcf', 'chr17_extracted.recode.vcf', 'chr10_extracted.recode.vcf', 'chr21_extracted.recode.vcf', 'chr12_extracted.recode.vcf'], returncode=0)

In [12]:
#Convert to binary plink
subprocess.run(["plink", "--vcf", "1000g.vcf.gz", "--make-bed", "--out", "1000Ghg19" ])
#Updating fam file
allfam = pd.read_csv("integrated_call_samples_v2.20130502.ALL.ped", header = None, skiprows = 1, sep = "\t")
oldfam = pd.read_csv("1000Ghg19.fam", header = None, sep = " ")
updatedfam = pd.merge(oldfam, allfam, how = "inner", left_on = 1, right_on = 1)
updatedfam.iloc[:,[6,1,7,8,9,5]].to_csv("1000Ghg19.fam", sep = " ", header = False, index = False)
os.remove("1000g.vcf.gz")

In [14]:
for file in glob.glob("chr*.recode.vcf"):
    os.remove(file)
    
dest_dir = os.path.join(projpath, "Results")
for filename in glob.glob("1000Ghg19.*"):
    shutil.copy(filename, dest_dir)

## Merge reference samples

Now we will merge the 1000G and HGDP databases, both using the hg19 coordinates and with related people removed.
We will attempt an initial merge, and then flip strands for the problematic SNPs. 
Then we will try a second attempt, and remove the still problematic snps from both databases, to then attempt the final merge.

In [16]:
os.chdir(os.path.join(projpath, "Results"))
#Preliminary merge
subprocess.run(["plink", "--bfile", "1000Ghg19", "--bmerge", "hgdp940hg19", "--make-bed", "--out", "hgdp1000ghg19"])
#Flip snps
subprocess.run(["plink", "--bfile", "1000Ghg19", "--flip", "hgdp1000ghg19-merge.missnp", "--make-bed", "--out", "1000Ghg19_flip"])
subprocess.run(["plink", "--bfile", "1000Ghg19_flip", "--bmerge", "hgdp940hg19", "--make-bed", "--out", "hgdp1000ghg19"])
#Removing snps
for file in glob.glob("*.bed"):
    outname = file.split(".")[0] + "_temp"
    subprocess.run(["plink", "--bfile", file.split(".")[0], "--exclude", "hgdp1000ghg19-merge.missnp", "--make-bed", "--out", outname])

#Removing temp files
subprocess.run(["plink", "--bfile", "hgdp940hg19_temp", "--bmerge", "1000Ghg19_flip_temp", "--make-bed", "--out", "hgdp1000ghg19"])
for file in glob.glob("*_temp*"):
    os.remove(file)
    
for file in glob.glob("*_flip*"):
    os.remove(file)

with open("hgdp1000ghg19.log", 'r') as fin:
    file_contents = fin.read()
    print(file_contents)

PLINK v1.90b4 64-bit (20 Mar 2017)
Options in effect:
  --bfile hgdp940hg19_temp
  --bmerge 1000Ghg19_flip_temp
  --make-bed
  --out hgdp1000ghg19

Hostname: tomasgazelle
Working directory: /home/tomas/Documents/HGDP_1000G_Merge/Results
Start time: Thu May 17 09:36:00 2018

Random number seed: 1526564160
3865 MB RAM detected; reserving 1932 MB for main workspace.
940 people loaded from hgdp940hg19_temp.fam.
2504 people to be merged from 1000Ghg19_flip_temp.fam.
Of these, 2504 are new, while 0 are present in the base dataset.
644003 markers loaded from hgdp940hg19_temp.bim.
640456 markers to be merged from 1000Ghg19_flip_temp.bim.
Of these, 0 are new, while 640456 are present in the base dataset.
Performing single-pass merge (3444 people, 644003 variants).
Merged fileset written to hgdp1000ghg19-merge.bed + hgdp1000ghg19-merge.bim +
hgdp1000ghg19-merge.fam .
644003 variants loaded from .bim file.
3444 people (1851 males, 1593 females) loaded from .fam.
Using 1 thread (no multithreaded c